In [3]:
import pandas as pd
import json
import folium
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm
import time

In [8]:
Susansijang = webdriver.Chrome(service=Service("/home/han/Desktop/Han_ws/00.Data/01.EDA/driver/chromedriver"))
Susansijang.maximize_window()
Susansijang.get("https://www.susansijang.co.kr/nsis/mim/info/mim9030")
Susansijang.implicitly_wait(5)

In [36]:
specie_name_list = Susansijang.find_element(By.ID,"fishing_species")
specie_names = specie_name_list.find_elements(By.TAG_NAME,"option")
species = [specie.text for specie in specie_names]

In [52]:
auction_box = Susansijang.find_element(By.TAG_NAME,"tbody")
auction_row = auction_box.find_elements(By.TAG_NAME,"tr")
auction = auction_row[0].text
auction_list = [auction.text for auction in auction_row]
auction_list

['(활)갑오징어 서천 1미 미(마리) 65 13,000 5,000 7,800',
 '(활)개조개 통영 14미 box 1 51,500 51,500 51,500',
 '(선)깐홍합 여수 소 box 63 10,500 5,500 7,500',
 '(선)꼴뚜기 여수 10봉 봉지 110 8,800 7,800 8,200',
 '(활)농어 격포 중 kg 87.6 18,000 1,500 15,000',
 '(활)돌게 태안 중 kg 19 5,600 5,600 5,600',
 '(활)민어 완도 중 kg 209.9 37,000 17,000 27,200',
 '(선)병어 완도 1미 미(마리) 23 4,000 2,500 3,300',
 '(활)보리새우 통영 1미 kg 5 10,000 10,000 10,000',
 '(선)복어 안흥 20미 S/P 8.6 3,000 3,000 3,000',
 '(냉)아귀 중국 5미 CT/(BT) 30 30,800 30,800 30,800',
 '(활)암꽃게 태안 중소 kg 289 12,000 11,700 12,000',
 '(냉)염고등어 노르웨이 대 CT/(BT) 100 36,500 36,500 36,500',
 '(활)우럭조개 여수 6미 미(마리) 18 10,700 10,000 10,500',
 '(선)은갈치 제주도 11미 S/P 1 90,000 90,000 90,000',
 '(선)은갈치 제주도 6미 S/P 5 246,000 242,000 242,800',
 '(활)전복 완도 22미 kg 12 12,500 12,500 12,500',
 '(활)전복 완도 10/11미 kg 333.4 20,500 15,000 16,100',
 '(선)참가자미 안흥 27미 S/P 1 50,000 50,000 50,000',
 '(선)참돔 서천 중 kg 78.1 10,000 8,000 9,200']

In [ ]:
sido = gasStation.find_element(By.ID,"SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME,"option")
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names = sido_names[1:]
sido.send_keys(sido_names[0])

sigungu = gasStation.find_element(By.ID, "SIGUNGU_NM0")   #select seoul
sigungu_list = sigungu.find_elements(By.TAG_NAME,"option")
sigungu_names = [sigungu_name.get_attribute("value") for sigungu_name in sigungu_list]
sigungu_names = sigungu_names[1:]

station_info_list = []

# district
for sigungu_name in tqdm(sigungu_names) :
    
    sigungu = gasStation.find_element(By.ID, "SIGUNGU_NM0")
    sigungu.send_keys(sigungu_name)
    gasStation.implicitly_wait(1)

    # number of gas stations
    station_count = gasStation.find_element(By.ID,"totCnt")
    station_count = int(station_count.text) 
    
    # body1 : gasoline, body2 : premium gasoline, body3 : disel, body4 : paraffin
    gasoline = gasStation.find_element(By.ID,"body1")       
    station_list= gasoline.find_elements(By.TAG_NAME, "a") 

    for count in range(station_count): 
    
        station_list[count].click()                                          

        # information gathering
        station_name    = gasStation.execute_script(f'return document.getElementById("{"os_nm"}").innerHTML;')                     # name
        station_number  = gasStation.execute_script(f'return document.getElementById("{"phn_no"}").innerHTML;')                    # number
        station_address = gasStation.execute_script(f'return document.getElementById("{"rd_addr"}").innerHTML;').split(maxsplit=2) # address
        station_brand   = gasStation.execute_script(f'return document.getElementById("{"poll_div_nm"}").innerHTML;')               # brand
        station_sigungu = station_address[1]
        station_address = station_address[2]

        station_price = [                                                                               # price
            gasStation.execute_script(f'return document.getElementById("{"b034_p"}").innerHTML;'),      # premium gasoline
            gasStation.execute_script(f'return document.getElementById("{"b027_p"}").innerHTML;'),      # gasoline
            gasStation.execute_script(f'return document.getElementById("{"d047_p"}").innerHTML;'),      # disel
            gasStation.execute_script(f'return document.getElementById("{"c004_p"}").innerHTML;')       # paraffin
        ]

        # popup detail information page
        station_detail_info = gasStation.find_element(By.ID,"os_dtail_info")    

        # additional service car wash, charging, maintenance, convenience, 24/7
        service_list = ["cwsh_yn", "lpg_yn", "maint_yn", "cvs_yn", "sel24_yn"]
        station_service = ["X"] * len(service_list)

        for service in service_list :                                           
            if 'off' not in station_detail_info.find_element(By.ID,service).get_attribute("src"):   station_service[service_list.index(service)]="O"

        # data merge 
        station_info = [station_sigungu, station_brand, station_name, station_address, station_number] 
        station_info.extend(station_price)
        station_info.extend(station_service)
        station_info_list.append(station_info)


In [53]:
Susansijang.quit()